## Importing required libraries

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

## Load Data

In [5]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|                                                                 | 0/1 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 647 examples [00:01, 643.77 examples/s]
Generating train examples...: 1291 examples [00:02, 630.12 examples/s]
Generating train examples...: 1982 examples [00:03, 653.64 examples/s]
Generating train examples...: 2697 examples [00:04, 677.48 examples/s]
Generating train examples...: 3550 examples [00:05, 740.21 examples/s]
                                                                      
Shuffling C:\Users\Durgesh\tensorflow_datasets\tf_flowers\3.0.1.incompleteLO997T\tf_flowers-train.tfrecord*...:   0%| |
Shuffling C:\Users\Durgesh\tensorflow_datasets\tf_flowers\3.0.1.incompleteLO997T\tf_flowers-train.tfrecord*...:  17%|▏|
Shuffling C:\Users\Durgesh\tensorflow_datasets\tf_flowers\3.0.1.incompleteLO997T\tf_flowers-train.tfrecord*...: 

Dataset tf_flowers downloaded and prepared to C:\Users\Durgesh\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


## Image Preprocessing

In [6]:
## check existing image size
train_ds[0].shape

TensorShape([442, 1024, 3])

In [7]:
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [26]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [8]:
train_labels

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0], dtype=int64)>

In [9]:
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [10]:
train_labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

### Use Pretrained VGG16 Image Classification model

# **Load a pre-trained CNN model trained on a large dataset**

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [12]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [13]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)

58889256/58889256 [==============================] - 12s 0us/step


In [14]:
## will not train base mode
# Freeze Parameters in model's lower convolutional layers
base_model.trainable = False 

In [15]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [16]:
## model details
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

**Add custom classifier with two dense layers of trainable parameters to model**

In [17]:
#add our layers on top of this model
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

**Train classifier layers on training data available for task**

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [19]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

In [21]:
history=model.fit(train_ds, train_labels, epochs=1, validation_split=0.2, batch_size=32, callbacks=[es])

65/65 [==============================] - 303s 5s/step - loss: 0.2285 - accuracy: 0.9280 - val_loss: 1.1758 - val_accuracy: 0.6615


In [22]:
los,accurac=model.evaluate(test_ds,test_labels)
print("Loss: ",los,"Accuracy: ", accurac)

35/35 [==============================] - 100s 3s/step - loss: 0.1404 - accuracy: 0.9628
Loss:  0.1404050886631012 Accuracy:  0.9627611041069031
